how this data contribution expands the OBIS holdings

- picking a species from SCCOOS, then pulling all the data for that species from OBIS
- plotting it all together with SCCOOS a different color?

In [1]:
library(robis)

sccoos_data <- occurrence(
    datasetid="c9aaa0e9-8f6c-4553-a014-a857baba0680"
)

Retrieved 5000 records of approximately 10483 (47%)
Retrieved 10000 records of approximately 10483 (95%)
Retrieved 10483 records of approximately 10483 (100%)


In [2]:
map_leaflet(sccoos_data)

HTML widgets cannot be represented in plain text (need html)